In [1]:
import pathlib

import numpy as np
import pandas as pd

In [2]:
# set paths
# input data
cls_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/test_run/CLS_features/channel_DNA_channel488-1_channel488-2_channel561_blank_model_sc-ViT_checkpoint0100_vitsmall16_features.csv"
).resolve(strict=True)
image_paths_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/test_run/CLS_features/image_paths.csv"
).resolve(strict=True)

# output path for the merged table
output_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/test_run/CLS_features/CLS_features_annotated.parquet"
).resolve()

In [3]:
# load data
cls_df = pd.read_csv(cls_path, header=None)
image_paths_df = pd.read_csv(image_paths_path, header=None)

In [4]:
print(len(cls_df), len(image_paths_df))

193146 193146


In [5]:
# rename each column to have cls_ prefix
cls_df.columns = [f"cls_{i}" for i in range(cls_df.shape[1])]
# rename the columns
image_paths_df.columns = ["image_paths"]

In [6]:
# combine data
cls_df["Metadata_image_path"] = image_paths_df["image_paths"]
cls_df.head(1)

,cls_0,cls_1,cls_2,cls_3,cls_4,cls_5,cls_6,cls_7,cls_8,cls_9,...,cls_375,cls_376,cls_377,cls_378,cls_379,cls_380,cls_381,cls_382,cls_383,Metadata_image_path
0,0.031278,-0.054149,0.107055,0.002328,0.034356,-0.048974,0.037127,-0.014773,0.026619,0.037709,...,-0.174261,-0.020648,0.015223,0.041441,-0.055121,0.038324,-0.046185,0.089597,0.006296,/home/lippincm/Documents/4TB/data/live_cell_ti...


In [7]:
# split column into multiple columns
cls_df["Metadata_Well"] = cls_df["Metadata_image_path"].apply(
    lambda x: pathlib.Path(x).name.split("_")[0]
)
cls_df["Metadata_FOV"] = cls_df["Metadata_image_path"].apply(
    lambda x: pathlib.Path(x).name.split("_")[1]
)
cls_df["Metadata_Time"] = cls_df["Metadata_image_path"].apply(
    lambda x: pathlib.Path(x).name.split("_")[2]
)
cls_df["Metadata_Channel"] = cls_df["Metadata_image_path"].apply(
    lambda x: pathlib.Path(x).name.split("_")[4]
)
cls_df["Metadata_Cell_id"] = cls_df["Metadata_image_path"].apply(
    lambda x: pathlib.Path(x).name.split("_")[6]
)
# move Metadata columns to the front
# pop out the Metadata columns
Metadata_cols = cls_df.columns[cls_df.columns.str.contains("Metadata")]
# move Metadata columns to the front
cls_df = cls_df[
    Metadata_cols.tolist() + cls_df.columns.difference(Metadata_cols).tolist()
]


cls_df.head()

,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_Channel,Metadata_Cell_id,cls_0,cls_1,cls_10,cls_100,...,cls_90,cls_91,cls_92,cls_93,cls_94,cls_95,cls_96,cls_97,cls_98,cls_99
0,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,100,0.031278,-0.054149,0.032339,0.016466,...,0.103771,-0.002917,-0.088505,-0.054172,0.009552,-0.053764,0.092698,0.052247,0.047284,0.025616
1,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,101,0.031099,-0.064783,0.014539,-0.002247,...,0.082508,-0.004323,-0.041911,-0.060461,0.007774,-0.065580,0.110302,-0.000034,0.024379,0.019590
2,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,102,0.038233,-0.058997,0.013999,-0.007888,...,0.090952,-0.004582,-0.037789,-0.048010,0.004603,-0.058200,0.084165,0.034247,0.050790,0.051865
3,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,103,0.037757,-0.071355,0.003984,0.007385,...,0.085263,0.001482,-0.084153,-0.051854,-0.000954,-0.049379,0.102376,0.030741,0.029512,0.040069
4,/home/lippincm/Documents/4TB/data/live_cell_ti...,C-02,F0001,T0001,C01,104,0.025290,-0.051070,0.019076,-0.002388,...,0.098721,-0.009961,-0.039805,-0.040878,0.022797,-0.054397,0.045918,0.046192,0.052029,0.035139


In [8]:
# save the data
cls_df.to_parquet(output_path, index=False)